<a href="https://colab.research.google.com/github/pstansfeld/MemProtMD/blob/main/MemProtMD_Self_Assembly.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
#@title Install dependencies
%%capture
import os
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')

if not os.path.isdir("/content/memembed/"):
  !apt-get update -y
  !git clone https://github.com/timnugent/memembed
  %cd memembed
  !make
  %cd ../
  !python3 -m pip install pdb2pqr
  !python3 -m pip install vermouth
  !python3 -m pip install GromacsWrapper
  !python3 -m pip install MDAnalysis 
  !python2 -m pip install insane
  !python3 -m pip install py3dmol
  !wget http://cgmartini.nl/images/martini_v300.zip
  !unzip -o martini_v300.zip
  !git clone https://github.com/owenvickery/cg2at
#  !wget https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh
#  !chmod +x Miniconda3-py37_4.8.2-Linux-x86_64.sh
#  !bash ./Miniconda3-py37_4.8.2-Linux-x86_64.sh -b -f -p /usr/local
#  !conda install -c conda-forge -c bioconda gromacs=2021.3 --yes
  !wget https://raw.githubusercontent.com/pb3lab/ibm3202/master/software/gromacs.tar.gz
  !tar xzf gromacs.tar.gz
  !ln -s /content/gromacs/bin/gmx /usr/bin/gmx

import gromacs
import MDAnalysis
import py3Dmol
import glob
import shutil
from google.colab import files

In [4]:
#@title Upload a PDB coordinate file
os.chdir('/content/')
upload = files.upload()
#@title Predict Membrane Orientation
mview = py3Dmol.view(800, 400)  
filename = next(iter(upload))
name = os.path.splitext(filename)[0]
working_dir = '/content/' + name + '/'
os.makedirs(working_dir, exist_ok=True)
os.rename(filename, working_dir + filename)
os.chdir(working_dir)
mol1 = open(working_dir + filename, 'r').read()
mview.addModel(mol1,'pdb')
mview.setStyle({'cartoon': {'color':'spectrum'}})
mview.setStyle({'resn':'DUM'},{'sphere':{}})
mview.setBackgroundColor('0xffffff')
mview.zoomTo()
mview.show()

Saving model_01.pdb to model_01.pdb


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [5]:
#@title Predict Membrane Orientation
os.chdir(working_dir)
os.system('/content/memembed/bin/memembed -o memembed.pdb ' + filename) 
mview = py3Dmol.view(800, 400)  
mol1 = open(working_dir + '/memembed.pdb', 'r').read()
mview.addModel(mol1,'pdb')
mview.setStyle({'cartoon': {'color':'spectrum'}})
mview.setStyle({'resn':'DUM'},{'sphere':{}})
mview.setBackgroundColor('0xffffff')
mview.zoomTo()
mview.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [6]:
#@title Set-up Coarse-Grained Membrane Protein System
%%capture
os.chdir(working_dir)

for file in glob.glob(r'/content/martini*.itp'):
    print(file)
    shutil.copy(file, working_dir)

gromacs.make_ndx(f='memembed.pdb', o='index.ndx', input=('del 0', 'del 1-100','rDUM','q'),backup=False)
gromacs.editconf(f='memembed.pdb',o='box.pdb',n='index.ndx',c=True,d=3,input=(0,0),backup=False)

u = MDAnalysis.Universe('box.pdb')
x = round(u.dimensions[0]/10)
y = round(u.dimensions[1]/10)
z = round(u.dimensions[2]/10)
smallbox = [x,y,8]
bigbox = [x,y,z]

gromacs.editconf(f='memembed.pdb',o='centered.pdb',label='A',resnr=1,n='index.ndx',c=True,input=(1,0),backup=False,box=smallbox)

with open('centered.pdb', 'r') as file :
  filedata = file.read()
filedata = filedata.replace('HSE', 'HIS')
filedata = filedata.replace('HSD', 'HIS')
filedata = filedata.replace('MSE', 'MET')
filedata = filedata.replace(' SE ', ' SD ')
with open('centered.pdb', 'w') as file:
  file.write(filedata)

!martinize2 -f centered.pdb  -ff martini3001 -x protein-cg.pdb -o protein-cg.top -elastic -ef 500 -eu 1.0 -el 0.5 -ea 0 -ep 0 -merge A -maxwarn 1000

!sed -e 's/^molecule.*/Protein 1/g' molecule*.itp >  protein-cg.itp

!wget 'http://cgmartini.nl/images/parameters/lipids/PC/POPC/POPC-em.gro'
!wget 'http://cgmartini.nl/images/applications/water/water.gro'

with open('topol.top','w') as top:
            top.write('#include "martini_v3.0.0.itp"\n#include "martini_v3.0.0_ions_v1.itp"\n#include "martini_v3.0.0_solvents_v1.itp"\n#include "martini_v3.0.0_phospholipids_v1.itp"\n#include "protein-cg.itp"\n[ system ]\n[ molecules ]\nProtein 1')

nlipid = round(x * y * 2.5)

gromacs.editconf(f='protein-cg.pdb', o='protein-box.pdb',label='A',input=(0),backup=False,box=smallbox,noc=True)

gromacs.insert_molecules(f='protein-box.pdb',ci='POPC-em.gro',o='prot+lipid.pdb',nmol=nlipid,radius=0.21,rot='xyz')

v = MDAnalysis.Universe('prot+lipid.pdb')
popc = v.select_atoms('name PO4')
po = popc.atoms.n_atoms

with open('topol.top','w') as top:
            top.write('#include "martini_v3.0.0.itp"\n#include "martini_v3.0.0_ions_v1.itp"\n#include "martini_v3.0.0_solvents_v1.itp"\n#include "martini_v3.0.0_phospholipids_v1.itp"\n#include "protein-cg.itp"\n[ system ]\n[ molecules ]\nProtein 1\nPOPC ' + str(po))

gromacs.editconf(f='prot+lipid.pdb',o='prot+lipid-box.pdb',c=True,box=bigbox)
gromacs.solvate(cp='prot+lipid-box.pdb',o='prot+lipid+sol.pdb',cs='water.gro',radius=0.21,backup=False,p='topol.top')

v = MDAnalysis.Universe('prot+lipid+sol.pdb')
wat = v.select_atoms('name W')
w = wat.atoms.n_atoms

print(w)

with open('topol.top','w') as top:
            top.write('#include "martini_v3.0.0.itp"\n#include "martini_v3.0.0_ions_v1.itp"\n#include "martini_v3.0.0_solvents_v1.itp"\n#include "martini_v3.0.0_phospholipids_v1.itp"\n#include "protein-cg.itp"\n[ system ]\n[ molecules ]\nProtein 1\nPOPC ' + str(po) + '\nW ' + str(w))

with open('em.mdp','w') as em:
            em.write('integrator = steep\nnsteps = 5000\nemtol = 1000\nemstep = 0.001')

gromacs.grompp(f='em.mdp',o='ion.tpr',c='prot+lipid+sol.pdb',maxwarn='-1',backup=False,v=True)
gromacs.genion(s='ion.tpr',o='CG-system.gro',neutral=True,conc=0.15,backup=False,input=str(14),p='topol.top')

v = MDAnalysis.Universe('CG-system.gro')
wat = v.select_atoms('name W')
sod = v.select_atoms('name NA')
chl = v.select_atoms('name CL')
w = wat.atoms.n_atoms
na = sod.atoms.n_atoms
cl = chl.atoms.n_atoms

with open('topol.top','w') as top:
            top.write('#include "martini_v3.0.0.itp"\n#include "martini_v3.0.0_ions_v1.itp"\n#include "martini_v3.0.0_solvents_v1.itp"\n#include "martini_v3.0.0_phospholipids_v1.itp"\n#include "protein-cg.itp"\n[ system ]\n[ molecules ]\nProtein 1\nPOPC ' + str(po) + '\nW ' + str(w) + '\nNA ' + str(na) + '\nCL ' + str(cl))

gromacs.grompp(f='em.mdp',o='em.tpr',c='CG-system.gro',maxwarn='-1',backup=False,v=True)

gromacs.mdrun(deffnm='em', c='CG-system.pdb',backup=False)

gromacs.trjconv(f='CG-system.pdb', o='CG-system.pdb', pbc='res', s='em.tpr', conect=True, input='0',backup=False)

%mkdir MD

gromacs.make_ndx(f='CG-system.pdb', o='sys.ndx', input=('del 0', 'del 1-40', '0|rPOP*','1&!0','!1','del 1','name 1 Lipid','name 2 SOL_ION','q'),backup=False)

with open('cgmd.mdp','w') as md:
            md.write('integrator = md\ntinit = 0.0\ndt = 0.02\nnsteps = 250000000\nnstxout = 0\nnstvout = 0\nnstfout = 0\nnstlog = 50000\nnstenergy = 50000\nnstxout-compressed = 50000\ncompressed-x-precision = 10000\nnstlist  = 10\nns_type  = grid\npbc   = xyz\ncoulombtype  = Reaction_field\nrcoulomb_switch = 0.0\nrcoulomb  = 1.1\nepsilon_r  = 15\nvdw_type  = cutoff\nrvdw_switch  = 0.9\nrvdw   = 1.1\ncutoff-scheme = verlet\ncoulomb-modifier = Potential-shift\nvdw-modifier  = Potential-shift\nepsilon_rf  = 0\nverlet-buffer-tolerance = 0.005\ntcoupl  = v-rescale\ntc-grps  = PROTEIN LIPID SOL_ION\ntau_t  = 1.0 1.0 1.0\nref_t  = 310 310 310\nPcoupl  = c-rescale\nPcoupltype  = semiisotropic\ntau_p  = 12.0\ncompressibility = 3e-4 3e-4\nref_p  = 1.0 1.0\ngen_vel  = yes\ngen_temp  = 310\ngen_seed  = -1\nconstraints  = none\nconstraint_algorithm = Lincs\ncontinuation  = no\nlincs_order  = 4\nlincs_warnangle = 30\n')

gromacs.grompp(f='cgmd.mdp',o='MD/md',c='CG-system.pdb',maxwarn=-1, n='sys.ndx',backup=False)  

mview = py3Dmol.view(800, 400)  
mol1 = open(working_dir + 'CG-system.pdb', 'r').read()
mview.addModel(mol1,'gro')
mview.setStyle({'cartoon': {'color':'spectrum'}})
mview.setStyle({'resn':'POPC'},{'sphere':{}})
mview.setStyle({'atom':'BB'},{'sphere':{}})
mview.setBackgroundColor('0xffffff')
mview.zoomTo()
mview.show()

for file in glob.glob(r'#*'):
    os.remove(file)
    
os.chdir(working_dir)
for file in glob.glob(r'#*'):
    os.remove(file)

In [ ]:
#@title Equilibrate Membrane Protein System
os.chdir(working_dir + 'MD')
gromacs.mdrun(deffnm='md',backup=False, v=True, nsteps=25000000,resethway=True,nstlist=100)
mview = py3Dmol.view(800, 400)  
mol1 = open(working_dir + 'MD/md.gro', 'r').read()
mview.addModel(mol1,'gro')
mview.setStyle({'cartoon': {'color':'spectrum'}})
mview.setStyle({'atom':'PO4'},{'sphere':{}})
mview.setStyle({'atom':'BB'},{'sphere':{}})
mview.setBackgroundColor('0xffffff')
mview.zoomTo()
mview.show()

for file in glob.glob(r'#*'):
    os.remove(file)
    
os.chdir(working_dir)
for file in glob.glob(r'#*'):
    os.remove(file)

In [ ]:
#@title Convert back to Atomistic
os.chdir(working_dir + 'MD/')
shutil.rmtree('CG2AT', ignore_errors=True)
os.system('/content/cg2at/cg2at -o align -w tip3p -c md.gro -a ../centered.pdb -loc CG2AT -ff charmm36-jul2020-updated -fg martini_3-0_charmm36')
os.rename(working_dir + 'MD/CG2AT/FINAL/final_cg2at_aligned.pdb',working_dir + name + '-System.pdb')
mview = py3Dmol.view(800, 400)  
mol1 = open(working_dir + name + '-System.pdb', 'r').read()
mview.addModel(mol1,'pdb')
mview.setStyle({'cartoon': {'color':'spectrum'}})
mview.setStyle({'atom':'P'},{'sphere':{}})
mview.setBackgroundColor('0xffffff')
mview.zoomTo()
mview.show()